In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from tqdm import tqdm 
sns.set_style("darkgrid")
from datetime import datetime
import json
import matplotlib as mpl
import time

In [2]:
data_path = "data/"
places = pd.read_csv(data_path + "places_final.csv", sep=",")
reviews = pd.read_csv(data_path + "reviews_cleaned.csv", sep=",")

In [3]:
reviews.groupby('gPlusPlaceId').count()['gPlusUserId']

gPlusPlaceId
100000206364762084593    13
100000245405929509254     6
100000275303571693008     2
100000306573193639539     2
100000307388846212836     2
                         ..
Sep 8, 2011               0
Sep 8, 2013               0
Sep 9, 2011               0
Sep 9, 2012               0
Sep 9, 2013               0
Name: gPlusUserId, Length: 103530, dtype: int64

**virker til der er noget der er lidt off ift places i reviews dataet!!**

## Datafram preperation

Counting all business per cell

In [4]:
df_grid=places.groupby('Grid')['gPlusPlaceId'].count().reset_index()
df_grid.rename(columns = {'gPlusPlaceId':'PlaceCount'}, inplace = True)

Counting number of business per category per cell

In [5]:
cat_cols=places.groupby(['Grid','category']).count()['gPlusPlaceId'].reset_index()
cat_cols=pd.pivot_table(cat_cols, index='Grid', columns='category', values='gPlusPlaceId').fillna(0)

In [6]:
df_grid=df_grid.join(cat_cols,on='Grid')

In [7]:
df_grid.shape

(508, 14)

Finding number of reviews per cell

In [8]:
mergeddf=places.merge(reviews,on='gPlusPlaceId',how='outer')

In [9]:
mergeddf=mergeddf.dropna(subset=['name'])

In [10]:
grid_reviews = mergeddf.groupby('Grid').count()['gPlusUserId'].reset_index()

In [11]:
df_grid=df_grid.merge(grid_reviews,on='Grid')

In [12]:
df_grid.shape

(508, 15)

In [13]:
df_grid.head()

,Grid,PlaceCount,Accommodation,Bar,Cafe,Cultural,Education,Health,Other,Outdoors,Restaurant,Retail,Service,Wholesale,gPlusUserId
0,L0,8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,0.0,1.0,0.0,138
1,L1,10,0.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,2.0,0.0,2.0,1.0,32
2,L10,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,9
3,L100,74,12.0,6.0,6.0,0.0,1.0,4.0,16.0,0.0,17.0,4.0,7.0,1.0,929
4,L101,163,21.0,19.0,20.0,1.0,1.0,10.0,19.0,0.0,37.0,21.0,9.0,5.0,1436


Avg rating per cell, and avg review per cell after restaurants with less than 3 reviews are removed

In [14]:
mergeddf['rating'] = mergeddf['rating'].astype(float)

In [15]:
avg_reviews = mergeddf.groupby('Grid').mean()['rating'].reset_index()

In [16]:
df_grid=df_grid.merge(avg_reviews,on='Grid')

In [17]:
cat_rev = mergeddf.groupby(['Grid','category']).mean()['rating'].reset_index()

In [18]:
cat_val=[]

for i in range(len(cat_rev)):
    cat_val.append(cat_rev['category'][i]+'_rating')

cat_rev['cat_val']=cat_val

In [19]:
category_rating=pd.pivot_table(cat_rev, index='Grid', columns='cat_val', values='rating').fillna(0)

In [20]:
df_grid=df_grid.merge(category_rating,on='Grid')

In [21]:
df_grid

,Grid,PlaceCount,Accommodation,Bar,Cafe,Cultural,Education,Health,Other,Outdoors,...,Cafe_rating,Cultural_rating,Education_rating,Health_rating,Other_rating,Outdoors_rating,Restaurant_rating,Retail_rating,Service_rating,Wholesale_rating
0,L0,8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.000000,0.000000,0.000000,0.000000,4.000000,2.75,4.130435,0.000000,4.000000,0.000000
1,L1,10,0.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,...,5.000000,0.000000,0.000000,4.833333,4.800000,0.00,4.416667,0.000000,3.500000,5.000000
2,L10,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.000000,0.000000,4.000000,0.000000,4.200000,0.00,0.000000,5.000000,0.000000,0.000000
3,L100,74,12.0,6.0,6.0,0.0,1.0,4.0,16.0,0.0,...,3.971014,0.000000,4.333333,4.000000,3.779221,0.00,3.840361,4.700000,2.550000,4.500000
4,L101,163,21.0,19.0,20.0,1.0,1.0,10.0,19.0,0.0,...,3.673267,4.447368,5.000000,4.709677,4.114286,0.00,3.766968,3.686957,4.193548,3.727273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,NY86,110,1.0,15.0,5.0,3.0,4.0,3.0,9.0,0.0,...,4.185185,3.000000,3.586207,4.400000,4.055556,0.00,3.797251,4.073171,4.093750,3.625000
504,NY87,32,0.0,2.0,1.0,0.0,3.0,14.0,7.0,2.0,...,4.142857,0.000000,4.400000,3.567568,3.785714,3.75,4.375000,0.000000,5.000000,2.666667
505,NY88,5,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,...,0.000000,0.000000,5.000000,0.000000,4.000000,0.00,4.333333,0.000000,0.000000,0.000000
506,NY98,2,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000


Counting positive and negative reviews in a grid cell

In [28]:
positive=np.zeros(len(mergeddf))
negative=np.zeros(len(mergeddf))
neutral=np.zeros(len(mergeddf))

for i in range(len(mergeddf)):
    sensitiv=[mergeddf['posReviewPercent'][i],mergeddf['negReviewPercent'][i],mergeddf['midReviewPercent'][i]]
    sensitivmax=np.where(sensitiv==np.max(sensitiv))
    
    if 0 in sensitivmax:
        positive[i]=1
        
    if 1 in sensitivmax:
        negative[i]=1   
        
    if 2 in sensitivmax:
        neutral[i]=1
    else:
        0
        

<ipython-input-28-fb0b6f689007>:9: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if 0 in sensitivmax:
<ipython-input-28-fb0b6f689007>:12: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if 1 in sensitivmax:
<ipython-input-28-fb0b6f689007>:15: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if 2 in sensitivmax:


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [206]:
mergeddf['positive'] = positive
mergeddf['negative'] = negative
mergeddf['neutral'] = neutral

In [207]:
mergeddf['negative']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
279730    0.0
279731    0.0
279732    0.0
279733    0.0
279734    0.0
Name: negative, Length: 279735, dtype: float64

In [25]:
max([1,7,2])

7

In [194]:
sensitivity = mergeddf.groupby('Grid').sum()[['positive','negative']].reset_index()

KeyError: "None of [Index(['positive', 'negative'], dtype='object')] are in the [columns]"

In [130]:
df_grid=df_grid.merge(sensitivity,on='Grid')

In [131]:
df_grid.head()

,Grid,PlaceCount,Accommodation,Bar,Cafe,Cultural,Education,Health,Other,Outdoors,...,Education_rating_y,Health_rating_y,Other_rating_y,Outdoors_rating_y,Restaurant_rating_y,Retail_rating_y,Service_rating_y,Wholesale_rating_y,positive,negative
0,L0,8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.000000,0.000000,4.000000,2.75,4.130435,0.000000,4.000000,0.000000,71,17
1,L1,10,0.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,...,0.000000,4.833333,4.800000,0.00,4.416667,0.000000,3.500000,5.000000,23,3
2,L10,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,4.000000,0.000000,4.200000,0.00,0.000000,5.000000,0.000000,0.000000,7,1
3,L100,74,12.0,6.0,6.0,0.0,1.0,4.0,16.0,0.0,...,4.333333,4.000000,3.779221,0.00,3.840361,4.700000,2.550000,4.500000,404,113
4,L101,163,21.0,19.0,20.0,1.0,1.0,10.0,19.0,0.0,...,5.000000,4.709677,4.114286,0.00,3.766968,3.686957,4.193548,3.727273,705,121


In [153]:
int(np.where([1,8,6]==np.max([1,8,6]))[0])

1

In [195]:
mergeddf

,gPlusPlaceId,name,price,lat,lon,city,address,Grid,category,rating,reviewerName,reviewText,gPlusUserId,posReviewPercent,negReviewPercent,midReviewPercent
0,101742583391038750118,Carpo London,NaN,51.509499,-0.135762,London,"16 Piccadilly, London W1J 0DE, United Kingdom",L159,Retail,5.0,ΕΜΜΑΝΟΥΗΛ ΓΚΙΝΗΣ,Πραγματικό κόσμημα στο κέντρο του Λονδίνου. Τ...,100297355982082025569,0.000,0.000,1.000
1,101742583391038750118,Carpo London,NaN,51.509499,-0.135762,London,"16 Piccadilly, London W1J 0DE, United Kingdom",L159,Retail,5.0,Christina Paraskevaidou,"Great coffee, great location!",100608123799186604185,0.809,0.000,0.191
2,101742583391038750118,Carpo London,NaN,51.509499,-0.135762,London,"16 Piccadilly, London W1J 0DE, United Kingdom",L159,Retail,5.0,Khalid Alkhaldi,NaN,101616629626079848214,NaN,NaN,NaN
3,101742583391038750118,Carpo London,NaN,51.509499,-0.135762,London,"16 Piccadilly, London W1J 0DE, United Kingdom",L159,Retail,5.0,dimitris gryparis,"True love. Amazing coffee, chocolate and people!!",102594303475289928175,0.740,0.000,0.260
4,101742583391038750118,Carpo London,NaN,51.509499,-0.135762,London,"16 Piccadilly, London W1J 0DE, United Kingdom",L159,Retail,5.0,Peter Turner,Carpo at Piccadilly The best coffee in London ...,102611008836812497820,0.200,0.059,0.740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279730,115388159885161004940,Pixie Market,2.0,40.721412,-73.987996,New York,"100 Stanton St, Manhattan, NY 10002",NY46,Retail,4.0,Laura San Giacomo,I love this small store and the whole lower ea...,109802552186240793889,0.375,0.028,0.597
279731,115388159885161004940,Pixie Market,2.0,40.721412,-73.987996,New York,"100 Stanton St, Manhattan, NY 10002",NY46,Retail,3.0,Desmond Chow,Eryn brinié korean based but french inspired c...,111514876659997758230,0.415,0.000,0.585
279732,115388159885161004940,Pixie Market,2.0,40.721412,-73.987996,New York,"100 Stanton St, Manhattan, NY 10002",NY46,Retail,5.0,Marietta Ulacia,A very unique store in the Lower East Side. Of...,113328969208536682278,0.193,0.086,0.722
279733,115388159885161004940,Pixie Market,2.0,40.721412,-73.987996,New York,"100 Stanton St, Manhattan, NY 10002",NY46,Retail,4.0,Lara Ruth,Cute hip clothing by up and coming designers. ...,117072475110846017004,0.308,0.000,0.692
